In [1]:
import numpy as np
import random

In [15]:
class Product:
    # constructor
    def __init__(self, id_, price_list, margins_list):
        self.id_ = id_
        # possible choices of price
        self.price_list = price_list
        # actual price
        self.price = price_list[0]
        # index of the actual price
        self.idx = 0
        #margin for each price
        self.margins_list = margins_list
        self.margin = margins_list[0]

    # set the four new possible choices of the prices
    def set_new_price_list(self, price_list):
        self.price_list = price_list
        self.price = price_list[0]


    # change the actual price between one of the possible choice ordered with increasing prices
    def change_price(self, new_index):
        self.price = self.price_list[new_index]
        self.margin = self.margins_list[new_index]

    # increase by one step the price in the vector of possible prices
    def increase_price(self):
        if self.price == max(self.price_list):
            return
        else:
            self.price = self.price_list[self.idx + 1]
            self.margin = self.margins_list[self.idx + 1]
            self.idx += 1


    # decrease by one step the price in the vector of possible prices
    def decrease_price(self):
        if self.price == min(self.price_list):
            return
        else:
            self.price = self.price_list[self.idx - 1]
            self.margin = self.margins_list[self.idx - 1]
            self.idx -= 1

In [17]:
#Hp: the costs are 80% of the lower prices

P1 = Product(0,[10,13,16,19],[ 2.,  5.,  8., 11.])
P2 = Product(1,[20,23,26,29],[ 4.,  7., 10., 13.])
P3 = Product(2,[30,33,36,39],[ 6.,  9., 12., 15.])
P4 = Product(3,[40,43,46,49],[ 8., 11., 14., 17.])
P5 = Product(4,[50,53,56,59],[10., 13., 16., 19.])

products = [P1,P2,P3,P4,P5]

In [18]:
# superclass
class User:
    # constructor
    def __init__(self, primary):
        # for each product an user has a reservation price
        self.reservation_price = [] # 5x1
        # stores the id {0,1,2,3,4} of products clicked
        self.products_clicked = []
        # stores the id {0,1,2,3,4} of products bought
        self.cart = []
        # stores the quantities of items bought for each product
        self.quantities = []
        # graph with the influence probabilities between the products
        self.P = np.zeros((5,5))
        # primary product shown
        self.primary = primary # {0,1,2,3,4} 


    # graph with the click probabilities on the products 
    def generate_graph(self, distribution):
        graph = np.zeros((5,5))
        graph = distribution
        return graph
            


# Inheritance
# 3 classes of users:

# targeted buyer, lower probability to click other products after the first
class User0(User):
    def __init__(self, primary):
        User.__init__(self, primary)
        self.reservation_price = [15,25,35,45,55] + np.random.normal(1, scale=3, size=5) # average reservation price
        self.P = User.generate_graph(self, np.random.uniform(0, 0.5, size=(5,5))) #lower influence probabilities


# curious buyer, more variable budget, higher probability to click on other products
class User1(User):
    def __init__(self, primary):
        User.__init__(self, primary)
        self.reservation_price = [15,25,35,45,55] + np.random.normal(1, scale=5, size=5) # more variable reservation price
        self.P = User.generate_graph(self, np.random.uniform(0.2, 1, size=(5,5))) # higher influence probabilities


# buyer with higher budget 
class User2(User):
    def __init__(self, primary):
        User.__init__(self, primary)
        self.reservation_price = 10 + [15,25,35,45,55] + np.random.normal(1, scale=3, size=5) # higher reservation price
        self.P = User.generate_graph(self, np.random.uniform(0, 1, size=(5,5))) # more variable influence probabilities
        


In [19]:
# Each day we have a list of users who enter the website, distributed with respect to their classes
class Daily_Customers:
   # constructor
    def __init__(self):
        self.Users = []

    
    # which type of user has to be created (which class)
    def whichUser(self, binary_vector, primary):
      if np.sum(binary_vector==0):
          self.Users.append(User0(primary))
      elif np.sum(binary_vector==1):
          self.Users.append(User1(primary))    
      elif np.sum(binary_vector==2):
          self.Users.append(User2(primary))


    # generate new users for the day
    def UsersGenerator(self, num_users, binary_vector):
        #create "alpha_ratio * num_users" users, without the alpha_0 in competitor website
        users_per_product = np.random.multinomial(num_users, np.random.dirichlet(np.ones(5)))
        for i in range(len(users_per_product)):
            print(users_per_product[i], "users with primary product", i+1)
            for j in range(users_per_product[i]):
                self.whichUser(binary_vector, i) #i is the index of the primary product
                

In [20]:
# Example
Day_0 = Daily_Customers()
Day_0.UsersGenerator(1000, np.array([0,1]))

268 users with primary product 1
341 users with primary product 2
69 users with primary product 3
56 users with primary product 4
266 users with primary product 5


In [79]:
class E_commerce:
    # constructor
    def __init__(self):
        # list of products
        self.products = []
        # list of lists of users
        self.daily_users = []
        # dataset with the history of products visited by each user in each day
        self.time_history = []
        # amount of money spent by the users, to implement in future
        self.cash = 0 
        # probability that the user checks the second product
        self.lambda_ = 0.5
        #number of clicks per product
        self.graph = self.generate_graph(np.ones(5))
        #list of the total daily rewards for each day
        self.daily_rewards = []


    def set_lambda(self, new_lambda):
        self.lambda_ = new_lambda
    
    
    def add_product(self, product):
        self.products.append(product)
    
    
    def set_products(self, product_list):
        self.products = product_list

    # graph with the probabilities to see the products 
    # 1 for the first secondary slot, lambda for the second one
    def generate_graph(self, distribution):
        graph = np.zeros((5,5))
        for i in range(5):
          # secondary slots indexes (0,1,2,3,4,5)-{i=primary}
          j = np.random.choice([x for x in range(5) if x != i ],2, replace = False) 
          # probability to see the first slot = 1
          graph[i,j[0]] = distribution[i]
          # 1 * lambda 
          graph[i,j[1]] = distribution[i] * self.lambda_
        return graph
    

    # simulate a day of visits in the website
    def simulate_day(self, num_users, binary_vector):
        D = Daily_Customers()
        D.UsersGenerator(num_users, binary_vector)
        self.daily_users.append(D.Users)
        rewards_of_the_day = 0
        # store the visits of the day
        Day = []
        # for each user visit (each day we can change the prices: we have to implement it)
        for i in range(num_users):
            visit = self.visit(D.Users[i])
            Day.append(visit)
            #for each user compute reward from the cart (sum of the margins of the products bought)
            for k in range(np.size(D.Users[i].cart)):          
                rewards_of_the_day += self.products[D.Users[i].cart[k]].margin
            

        self.time_history.append(Day)
        self.daily_rewards.append(rewards_of_the_day)
        return Day
      


    # simulate when an user visits the website
    def visit(self, user):
        # Influence probability matrix of the products, for each user equal to the see probability*click probability
        prob_matrix = user.P * self.graph
        n_nodes = prob_matrix.shape[0]

        # if user's reservation price is lower than the price of the primary product -> end the visit
        if user.reservation_price[user.primary] < self.products[user.primary].price:
            user.products_clicked = [user.primary]
            history_purchase = []
            user.cart = []
            user.quantities = np.zeros(user.P.shape[0])
            active_nodes = np.zeros(n_nodes)
            active_nodes[user.primary] = 1
            return np.array([active_nodes])

        # Influence probability matrix of the products, for each user equal to the see probability*click probability
        prob_matrix = user.P * self.graph
        n_nodes = prob_matrix.shape[0]
        active_nodes = np.zeros(n_nodes)
        active_nodes[user.primary] = 1
        newly_active_nodes = active_nodes
        round = 0

        # store index of products clicked
        history_click = [user.primary] 
        # store products bought
        history_purchase = [user.primary] 
        # store products shown to the user but maybe not clicked or bought
        history_nodes = np.array([active_nodes]) 
        
        # store the prices of the 5 products in an array
        prod_prices = np.zeros(len(self.products))
        for i in range(len(self.products)):
            prod_prices[i] = self.products[i].price

        #user can't click again on products already bought
        prob_matrix[:, user.primary] = 0.

        while(round < 5 and np.sum(newly_active_nodes)>0):

            p = (prob_matrix.T * active_nodes).T
            products_clicked = p > np.random.rand(p.shape[0], p.shape[1])

            prob_matrix = prob_matrix * ((p!=0)==products_clicked)
            newly_active_nodes = (np.sum(products_clicked, axis=0)>0) * (1-active_nodes)

            #user can't click again on product already clicked
            prob_matrix[:, newly_active_nodes==1] = 0.
            
            # slots clicked
            secondary_slots = np.where(newly_active_nodes == 1)[0] 

            # check idxs which match the reservation price
            stop_idxs = np.where(np.array(user.reservation_price)[secondary_slots] < prod_prices[secondary_slots])
            go_idxs = np.where(np.array(user.reservation_price)[secondary_slots] >= prod_prices[secondary_slots])

            #users don't buy products higher than their reservation price
            #so the visit can't go on for theese idxs because no secondary products are shown, since the primary isn't bought
            prob_matrix[secondary_slots[stop_idxs],:] = 0.

            active_nodes = newly_active_nodes
            round += 1

            for i in range(np.size(np.where(newly_active_nodes==1)[0])):
                history_click.append(np.where(newly_active_nodes==1)[0][i])
            for i in range(np.size(go_idxs[0])):
                history_purchase.append(secondary_slots[go_idxs[0][i]])
            history_nodes = np.concatenate((history_nodes, [newly_active_nodes]),axis=0)

        # for better visualization of the array
        history_click =  history_click[:-1] 
        user.products_clicked = history_click
        history_purchase = history_purchase[:-1]
        user.cart = history_purchase

        # estimate random quantities for each product
        if(len(history_purchase)!=0):
            for i in range(len(history_purchase)):
                if(np.size(history_purchase[i])!=0):
                    for j in range(np.size(history_purchase[i])):
                        user.quantities.append(1 + np.random.randint(3, size = 1)[0]) # from 1 to 3 units bought
        #print(history_nodes)
        return history_nodes
    

In [73]:
#TEST VISIT FUNCTION

# users of the day
Day_test = Daily_Customers()
Day_test.UsersGenerator(1, np.array([0,0])) # 1 user of 1st class

#products
P1 = Product(0,[10,13,16,19],[ 2.,  5.,  8., 11.])
P2 = Product(1,[20,23,26,29],[ 4.,  7., 10., 13.])
P3 = Product(2,[30,33,36,39],[ 6.,  9., 12., 15.])
P4 = Product(3,[40,43,46,49],[ 8., 11., 14., 17.])
P5 = Product(4,[50,53,56,59],[10., 13., 16., 19.])
products = [P1,P2,P3,P4,P5]

#E_commerce inizialization
E = E_commerce()
E.set_products(products)
E.set_lambda(0.5)

U = Day_test.Users[0]
print(U.P * E.graph)
print("############################################################")
visit = E.visit(U)
print(visit)

0 users with primary product 1
0 users with primary product 2
1 users with primary product 3
0 users with primary product 4
0 users with primary product 5
[[0.         0.         0.05807205 0.30108146 0.        ]
 [0.36508567 0.         0.08421322 0.         0.        ]
 [0.15748558 0.07275162 0.         0.         0.        ]
 [0.0646095  0.         0.         0.         0.20350159]
 [0.         0.06012595 0.23962868 0.         0.        ]]
############################################################
[[0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0.]]


In [74]:
clicks = U.products_clicked
cart = U.cart
print(clicks)
print("***************************")
print(cart)
print("***************************")
print(U.quantities)

[2, 0, 3]
***************************
[2, 0, 3]
***************************
[2, 3, 2]


In [75]:
num_different_items_bought = len(U.quantities)
num_total_quantities = np.sum(U.quantities)

num_different_items_bought, num_total_quantities

(3, 7)

In [76]:
def estimate_probabilities(dataset, node_index, n_nodes):
    estimated_prob = np.ones(n_nodes)*1.0/(n_nodes-1)
    credits = np.zeros(n_nodes)
    occur_v_active = np.zeros(n_nodes)
    #n_episodes = len(dataset)
    for episode in dataset:
        idx_w_active = np.argwhere(episode[:, node_index] ==1).reshape(-1)
        if len(idx_w_active)>0 and np.any(idx_w_active>0):
            active_nodes_in_prev_step = episode[idx_w_active - 1, :].reshape(-1)
            credits += active_nodes_in_prev_step/np.sum(active_nodes_in_prev_step)
        for v in range(0,n_nodes):
            if(v!=node_index):
                idx_v_active = np.argwhere(episode[:, v] == 1).reshape(-1)
                if (len(idx_v_active)>0 and len(idx_w_active)==0) or (np.any(idx_v_active<idx_w_active) and len(idx_v_active)>0):
                    occur_v_active[v]+=1
    estimated_prob = np.nan_to_num(credits/occur_v_active)
    return estimated_prob

In [80]:
#TEST ESTIMATE PROBABILITIES

P1 = Product(0,[10,13,16,19],[ 2.,  5.,  8., 11.])
P2 = Product(1,[20,23,26,29],[ 4.,  7., 10., 13.])
P3 = Product(2,[30,33,36,39],[ 6.,  9., 12., 15.])
P4 = Product(3,[40,43,46,49],[ 8., 11., 14., 17.])
P5 = Product(4,[50,53,56,59],[10., 13., 16., 19.])
products = [P1,P2,P3,P4,P5]

#E_commerce inizialization
E = E_commerce()
E.set_products(products)
E.set_lambda(0.5)

num_users = 10000
binary_vector = np.array([0,1])


num_products = 5
n_nodes = num_products
node_index = 0
dataset = []

dataset = E.simulate_day(num_users, binary_vector)
for node_index in [0,1,2,3,4]:
  estimated_prob = estimate_probabilities(dataset=dataset, node_index=node_index, n_nodes=n_nodes) #estimate influence probabilities
  print("Estimated P Matrix: ", estimated_prob) # probability starting from node_index to reach other nodes


1566 users with primary product 1
263 users with primary product 2
391 users with primary product 3
284 users with primary product 4
7496 users with primary product 5


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in true_divide
  app.launch_new_instance()


Estimated P Matrix:  [0.         0.000251   0.21701113 0.         0.12058321]
Estimated P Matrix:  [0.24651568 0.         0.12040134 0.27816901 0.24720745]
Estimated P Matrix:  [0.12379032 0.11918063 0.         0.         0.00013051]
Estimated P Matrix:  [0. 0. 0. 0. 0.]
Estimated P Matrix:  [0.00181818 0.21875    0.00784929 0.12323944 0.        ]


In [81]:
E.daily_rewards

[31384.0]

In [ ]:
#STEP 3

In [90]:
class Environment:
  #take as input the E-commerce object with the prices already changed
  #and the day_index to access to the informations
    def __init__(self, n_arms, E_commerce, product_index, day_starting_index):
        self.n_arms = n_arms
        self.E = E_commerce
        self.day = day_starting_index
        self.product_idx = product_index

    def round(self, pulled_arm):
      #Reward is given by the simulation of a day in the E-commerce website
        self.E.products[self.product_idx].change_price(pulled_arm)
        reward = self.E.daily_rewards[self.day]
        self.day += 1
        return reward

In [85]:
class Learner:
    def __init__(self, n_arms):
        self.n_arms = n_arms
        self.t = 0
        self.rewards_per_arm = x = [[0] for i in range(n_arms)]
        self.collected_rewards = np.array([])

    def update_observations(self, pulled_arm, reward):
        self.rewards_per_arm[pulled_arm].append(reward)
        self.collected_rewards = np.append(self.collected_rewards, reward)

In [92]:
class TS_Learner(Learner):
    def __init__(self, n_arms):
        super().__init__(n_arms)
        self.beta_parameters = np.ones((n_arms, 2))

    def pull_arm(self):
        print(self.beta_parameters[:, 1])
        idx = np.argmax(np.random.beta(self.beta_parameters[:, 0], self.beta_parameters[:, 1]))
        return idx

    def update(self, pulled_arm, reward):
        self.t += 1
        self.update_observations(pulled_arm, reward)
        self.beta_parameters[pulled_arm, 0] = self.beta_parameters[pulled_arm, 0] + reward
        self.beta_parameters[pulled_arm, 1] = self.beta_parameters[pulled_arm, 1] + 1.0 - reward

In [93]:

ts_rewards_per_experiment = []

P1 = Product(0,[10,13,16,19],[ 2.,  5.,  8., 11.])
P2 = Product(1,[20,23,26,29],[ 4.,  7., 10., 13.])
P3 = Product(2,[30,33,36,39],[ 6.,  9., 12., 15.])
P4 = Product(3,[40,43,46,49],[ 8., 11., 14., 17.])
P5 = Product(4,[50,53,56,59],[10., 13., 16., 19.])
products = [P1,P2,P3,P4,P5]

#E_commerce inizialization
E = E_commerce()
E.set_products(products)
E.set_lambda(0.5)

num_users = 10000
binary_vector = np.array([0,1])


num_products = 5
n_nodes = num_products
n_arms = 4

T = 100 
dataset = []

dataset = E.simulate_day(num_users, binary_vector)

env = Environment(n_arms, E, 0, 0)
ts_learner = TS_Learner(n_arms=n_arms)
for t in range(0, T):
    # Thompson Sampling Learners
    pulled_arm = ts_learner.pull_arm()
    reward = env.round(pulled_arm)
    ts_learner.update(pulled_arm, reward)


    ts_rewards_per_experiment.append(ts_learner.collected_rewards)


292 users with primary product 1
5082 users with primary product 2
1040 users with primary product 3
2027 users with primary product 4
1559 users with primary product 5
[1. 1. 1. 1.]
[ 1. -3.  1.  1.]


ValueError: ignored

In [83]:
#Estimate conversion rates

P1 = Product(0,[10,13,16,19],[ 2.,  5.,  8., 11.])
P2 = Product(1,[20,23,26,29],[ 4.,  7., 10., 13.])
P3 = Product(2,[30,33,36,39],[ 6.,  9., 12., 15.])
P4 = Product(3,[40,43,46,49],[ 8., 11., 14., 17.])
P5 = Product(4,[50,53,56,59],[10., 13., 16., 19.])
P = np.array(([10,13,16,19],[20,23,26,29],[30,33,36,39],[40,43,46,49],[50,53,56,59]))

for z in range(0,4):
        products = [P1,P2,P3,P4,P5]
        products[z].increase_price()
        #E_commerce inizialization
        E = E_commerce()
        E.set_products(products)
        E.set_lambda(0.5)

        num_users = 10000
        binary_vector = np.array([0,1])


        num_products = 5
        n_nodes = num_products
        node_index = 0
        dataset = []

        conv_rates = np.zeros((5,4))

        dataset = E.simulate_day(num_users, binary_vector)
        clicks_per_product = np.array([0,0,0,0,0])
        purchases_per_product = np.array([0,0,0,0,0])

        for i in range(len(E.daily_users)):
            for j in range(len(E.daily_users[i])):
                for k in range(len(E.daily_users[i][j].products_clicked)):
                    for h in range(len(np.array([E.daily_users[i][j].products_clicked[k]]))):#lista h di prodotti clickati
                        if E.daily_users[i][j].products_clicked[h]==0:
                          clicks_per_product[0] += 1

                        elif E.daily_users[i][j].products_clicked[h]==1:
                          clicks_per_product[1] += 1

                        elif E.daily_users[i][j].products_clicked[h]==2:
                          clicks_per_product[2] += 1
                        
                        elif E.daily_users[i][j].products_clicked[h]==3:
                          clicks_per_product[3] += 1

                        elif E.daily_users[i][j].products_clicked[h]==4:
                          clicks_per_product[4] += 1

        print("clicks",clicks_per_product)


        for i in range(len(E.daily_users)):
            for j in range(len(E.daily_users[i])):
                for k in range(len(E.daily_users[i][j].cart)):
                    for h in range(np.size(E.daily_users[i][j].cart[k])):#lista h di prodotti clickati
                        if E.daily_users[i][j].products_clicked[h]==0:
                          purchases_per_product[0] += 1

                        elif E.daily_users[i][j].products_clicked[h]==1:
                          purchases_per_product[1] += 1

                        elif E.daily_users[i][j].products_clicked[h]==2:
                          purchases_per_product[2] += 1
                        
                        elif E.daily_users[i][j].products_clicked[h]==3:
                          purchases_per_product[3] += 1

                        elif E.daily_users[i][j].products_clicked[h]==4:
                          purchases_per_product[4] += 1

        print("purchases", purchases_per_product)   

        conv_rates[:,z] = np.divide(clicks_per_product,purchases_per_product)

conv_rates

3830 users with primary product 1
279 users with primary product 2
1176 users with primary product 3
877 users with primary product 4
3838 users with primary product 5
clicks [2359  156  647  488 1992]
purchases [1716  134  598  445 1758]
103 users with primary product 1
1378 users with primary product 2
964 users with primary product 3
417 users with primary product 4
7138 users with primary product 5
clicks [  61  765  469  196 3616]
purchases [  50  539  425  183 3132]
1301 users with primary product 1
62 users with primary product 2
2725 users with primary product 3
4382 users with primary product 4
1530 users with primary product 5
clicks [ 721   37 1515 2278  781]
purchases [ 442   24 1004 1879  667]
153 users with primary product 1
3157 users with primary product 2
2984 users with primary product 3
2638 users with primary product 4
1068 users with primary product 5
clicks [  73 1879 1615 1371  478]
purchases [  47 1164 1066  854  390]


array([[0.        , 0.        , 0.        , 1.55319149],
       [0.        , 0.        , 0.        , 1.61426117],
       [0.        , 0.        , 0.        , 1.51500938],
       [0.        , 0.        , 0.        , 1.60538642],
       [0.        , 0.        , 0.        , 1.22564103]])